# Create dataset from yolo's dataset

In [ ]:
!unzip "/content/egg detection label.v1i.yolov8.zip" -d "/content/egg-detection-label-yolov8"

In [4]:
import os
import cv2

dataset_path = "/content/egg-detection-label-yolov8"
image_dir = "/content/egg-detection-label-yolov8/train/images"
labels_dir = "/content/egg-detection-label-yolov8/train/labels"
output_dir = "/content/egg-cropped"

os.makedirs(output_dir, exist_ok=True)

for label_file in os.listdir(labels_dir):
  if not label_file.endswith(".txt"):
    continue

  image_name = label_file.replace(".txt", ".jpg")
  image_path = os.path.join(image_dir, image_name)

  if not os.path.exists(image_path):
    continue

  img = cv2.imread(image_path)
  h, w, _ = img.shape

  with open(os.path.join(labels_dir, label_file), "r") as f:
    for i, line in enumerate(f):
      class_id, x_center, y_center, width, height = map(float, line.strip().split())

      x_center *= w
      y_center *= h
      width *= w
      height *= h

      x1 = int(x_center - width / 2)
      y1 = int(y_center - height / 2)
      x2 = int(x_center + width / 2)
      y2 = int(y_center + height / 2)

      crop = img[y1:y2, x1:x2]

      if crop.size == 0:
        continue

      # create folder
      class_folder = os.path.join(output_dir, f"class{int(class_id)}")
      os.makedirs(class_folder, exist_ok=True)

      # save img
      output_path = os.path.join(class_folder, f"{image_name[:-4]}_{i}.jpg")
      cv2.imwrite(output_path, crop)

In [5]:
!mv "/content/egg-cropped/class0" "/content/egg-cropped/Damaged"
!mv "/content/egg-cropped/class1" "/content/egg-cropped/Not Damaged"

In [6]:
dataset_dir = '/content/egg-cropped'

classes = os.listdir(dataset_dir)
print(f"classes: {classes}")

for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    print(f"{cls}: {len(os.listdir(cls_path))} images")

classes: ['Damaged', 'Not Damaged']
Damaged: 728 images
Not Damaged: 177 images


In [ ]:
# Train the model by cropped image